In [2]:
import requests
import pandas as pd
import os
import numpy as np
state_names = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", "Connecticut", "District ", "of Columbia", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]

unique_committees = ['labor', 'rules', 'health', 'energy', 'budget', 'banking', 'science', 'finance', 'housing', 'commerce', 'pensions', 'forestry', 'nutrition', 'education', 'agriculture', 'publicworks', 'environment', 'urbanaffairs', 'armedservices', 'appropriations', 'transportation', 'foreignrelations', 'homelandsecurity', 'naturalresources', 'governmentalaffairs', 'jointcongressionalcommitteeoninaugural']
#^note, I copied this list after running the script before having this list, it can be re-created in this way with modifications to the code

url = 'https://www.senate.gov/committees/committee_assignments.htm'

import requests
from bs4 import BeautifulSoup
import PyPDF2

reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')
from pathlib import Path





urls = []
for link in soup.find_all('a'):
    if str(link.get('href'))[-3:]=="pdf":
        l = str(link.get('href'))
        if l[0]=='/':
            l="https://www.senate.gov/"+l
        response = requests.get(l)
        filename = Path('committee_pdfs/'+l.split('/')[-1]).write_bytes(response.content)

\nVerDate Aug 31 2005 03:07 Mar 15, 2006 Jkt 026433 PO 00000 Frm 00031 Fmt 6608 Sfmt 6608 E:\\HR\\OC\\26433.XXX 26433ycherry on PROD1PC64 with HEARING

\nVerDate Aug 31 2005 03:07 Mar 15, 2006 Jkt 026433 PO 00000 Frm 00030 Fmt 6608 Sfmt 6608 E:\\HR\\OC\\26433.XXX 26433ycherry on PROD1PC64 with HEARING

In [25]:
def dfFromPDF(pdf):
    pdfFileObj = open(pdf, 'rb')
    pdf = PyPDF2.PdfReader(pdfFileObj)

    text=""
    for page in pdf.pages:
        text=text+'\n'+page.extract_text().split('\nVerDate')[0]
    year = (pdf.pages[0].extract_text().split('WASHINGTON : 1')[-1].split(' S.')[0].split(' ')[2])

    # closing the pdf file object
    pdfFileObj.close()
    


    text=text.split('COMMITTEE ASSIGNMENTS OF SENATORS')[1]
    text=text.split('LIST OF SENATORS BY STATES ')[0]
    for i in range(10):
        text=text.replace(str(i), '')
    # text.replace('Mrs. ', ' WOMAN')

    

    # print(repr(text))
    text = text.replace('.','')
    
    
    text = text.replace(' (chairman)','')
    text=text.lower()
    for s in state_names:
        text=text.replace('of '+s.lower(),'')
    text = text.replace('  ',' ')
    text=text.replace('chairman','')
    text=text.replace('(','')
    text=text.replace(')','')
    entries = text.split(' \n')

    # entries = str(entries).replace('\n', '').replace("'",'').replace('[','').replace('[','').split(', ')

    member_list = []
    for e in range(len(entries)):
        member_dict = {}
        if(entries[e][0:2]=='mr' or entries[e][0:3]=='mrs'):
            

            #gender
            gender = 'M'
            if (entries[e][0:3]=='mrs'):
                gender='W'
            member_dict['gender'] = gender
            name = entries[e][3:].lower()

            #year
            member_dict['year']=int(year)

            #name
            if(name[0]==' '):
                name=name[1:]
            name=name.split(' ')[0]
            # name=name.replace('\n','')
            name=name.capitalize()
            member_dict['name'] = name

            #committees
            committees = entries[e][4:].split(' ',1)[1]#.replace(', and ', ', ').replace(' and ', ', ').split(', ')
            if committees[0:1]==' ':
                committees=committees[1:]
            for i in range(10):
                if committees[-1:]==' ':
                    committees=committees[0:-1]
            if committees[0:24]=='rules and administration':
                committees='rules and administration'
            if committees[0:14]=='appropriations':
                committees='appropriations'
            committees=str(committees).replace('hollen ','')
            committees=str(committees).replace('ccaskill ','')
            member_dict['committees'] = committees

            member_list.append(member_dict)

    df = pd.DataFrame(data=member_list)
    return df

# print(dfFromPDF('committee_pdfs/GPO-CPUB-109spub20.pdf'))

In [24]:
def test(pdf):
    pdfFileObj = open(pdf, 'rb')
    pdf = PyPDF2.PdfReader(pdfFileObj)

    
    year = (pdf.pages[0].extract_text().split('WASHINGTON : 1')[-1].split(' S.')[0].split(' ')[2])
    print(year)
    # closing the pdf file object
    pdfFileObj.close()


directory = 'committee_pdfs'
dfs = []
for filename in os.listdir(directory):
    if str(filename)[0:3]=='GPO':
        test('committee_pdfs/'+str(filename))

2006
2007
2007
2009
2009
2010
2010
2010
2009
2011
2014
2014
2013
2015
2018
2017
2019
2021


In [27]:


directory = 'committee_pdfs'
dfs = []
for filename in os.listdir(directory):
    if str(filename)[0:3]=='GPO':
        dfs.append(dfFromPDF('committee_pdfs/'+str(filename)))

df = pd.concat(dfs)
df.to_csv("committees.csv")
df.head()


,gender,year,name,committees
0,M,2006,Akaka,armed services
1,M,2006,Alexander,budget
2,M,2006,Allard,appropriations
3,M,2006,Allen,"commerce, science, and transportation"
4,M,2006,Baucus,"agriculture, nutrition, and forestry"


In [70]:
np.unique(df['committees'])

array(['agriculture, nutrition, and forestry', 'appropriations',
       'armed service', 'armed services',
       'banking, housing, and urban affairs', 'budget',
       'commerce, science, and transportation',
       'energy and natural resources', 'environment and public works',
       'finance', 'foreign relations',
       'health, education, labor, and pensions',
       'homeland security and governmental affairs',
       'joint congressional committee on inaugural',
       'rules and administration'], dtype=object)

In [29]:
equivalents = {
    'finance':'financial services',
    'agriculture, nutrition, and forestry':'agriculture',
    'budget':'budget',
    'appropriations':'appropriations',
    'armed service':'national security',
    'armed services':'national security',
    'banking, housing, and urban affairs':'banking and financial services', 
    'budget':,
    'commerce, science, and transportation':,
    'energy and natural resources':, 
    'environment and public works':,
    'finance', 'foreign relations':,
    'health, education, labor, and pensions':,
    'homeland security and governmental affairs':,
    'joint congressional committee on inaugural':,
    'rules and administration':
}

SyntaxError: invalid syntax (1823820931.py, line 10)

In [72]:
xls_unique_committees = ['deficit reduction', 'small business', 'financial services', 'natural resources', 'international relations', 'intelligence', 'resources', 'budget', 'ways and means', 'minority leader', 'standards of official conduct', 'science and technology', 'education and the workforce/workplace', 'armed services', 'energy independence and global warming', 'house oversight', 'investigate the voting irregularities of august 2, 2007', 'majority whip', 'house administration', 'energy and commerce', 'agriculture', 'printing', 'transportation and infrastructure', 'library', 'veterans affairs', 'banking and financial services', 'ethics', 'oversight and government reform', 'majority leader', 'education and labor', 'minority whip', 'national security', 'taxation', 'judiciary', 'homeland security', 'rules', 'government reform', 'speaker', 'foreign affairs', 'economic', 'appropriations']
print(xls_unique_committees)

['deficit reduction', 'small business', 'financial services', 'natural resources', 'international relations', 'intelligence', 'resources', 'budget', 'ways and means', 'minority leader', 'standards of official conduct', 'science and technology', 'education and the workforce/workplace', 'armed services', 'energy independence and global warming', 'house oversight', 'investigate the voting irregularities of august 2, 2007', 'majority whip', 'house administration', 'energy and commerce', 'agriculture', 'printing', 'transportation and infrastructure', 'library', 'veterans affairs', 'banking and financial services', 'ethics', 'oversight and government reform', 'majority leader', 'education and labor', 'minority whip', 'national security', 'taxation', 'judiciary', 'homeland security', 'rules', 'government reform', 'speaker', 'foreign affairs', 'economic', 'appropriations']


In [ ]:
df = pd.read_csv("committees.csv", usecols=['gender', 'year', 'name', 'committees'])
df['committees'] = df['committees'].str.replace("'",'').str.replace('"','').str.replace("[",'').str.replace("]",'').str.split(', ')
df.loc[0, 'committees']

C:\Users\benst\AppData\Local\Temp\ipykernel_30128\3078399620.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['committees'] = df['committees'].str.replace("'",'').str.replace('"','').str.replace("[",'').str.replace("]",'').str.split(', ')


['armedservices']

In [ ]:
def getCommitteesByName(name, min_year=2005, max_year=2023):
    list_of_lists = (df.loc[df[(df['name'].str.contains(name.lower().capitalize())) & (df['year']>=min_year) & (df['year']<=max_year)].index, 'committees'])
    if len(list_of_lists)==0:
        return []
    result = list(set((str(list_of_lists.sum()).replace("'",'').replace('[]',',').replace('[','').replace(']','').replace(', ',',').replace(' ','')).split(',')))
    return result

print(getCommitteesByName('Booker'))

unique_committees_series = (pd.Series(sorted(list({x for l in df['committees'] for x in l}),key=len)))
unique_committees_series=unique_committees_series[unique_committees_series.str.contains('mr')==False]
unique_committees=[]
for c in unique_committees_series:
    add=True
    if len(unique_committees)>0:
        
        for l in unique_committees:
            if l in c:
                # add=False
                print()
    if(add==True):
        unique_committees.append(c)

str(unique_committees)


NameError: name 'df' is not defined